In [1]:
from win32com.client import Dispatch

# df = land_data

xlApp = Dispatch("Excel.Application")
xlApp.Visible = 1
workbook_path = r'\\wsl.localhost\Ubuntu-22.04\home\flas\source\course\modelling\agristrat\附件1.xlsx'
xlApp.Workbooks.Open(workbook_path)

<COMObject Open>

In [ ]:

sheet_name = 'land' #Make sure this sheet exists!
sheet = xlApp.ActiveWorkbook.Sheets(sheet_name)
sheet.select
sheet.cells.clearcontents # keep as lowecase letters. Camel case doesn't work?!
y, x = 1 + df.shape[0], df.shape[1]
sheet.Range(sheet.Cells(1, 1), sheet.Cells(1, x)).Value = list(df.columns)
sheet.Range(sheet.Cells(2, 1), sheet.Cells(y, x)).Value = list(df.values)

xlApp.ActiveWorkbook.RefreshAll #Refresh Pivot Tables if any
xlApp.ActiveWorkbook.Save

# send_email = False
# if send_email == True: xlApp.Application.Run('Python_SendEmail') # In case you wanted to run vba macro
# xlApp.ActiveWorkbook.Close(SaveChanges=True)
# xlApp.Quit()
# del xlApp


In [ ]:
# Copyright 2024, Gurobi Optimization, LLC

# This example formulates and solves the following simple MIP model:
#  maximize
#        x +   y + 2 z
#  subject to
#        x + 2 y + 3 z <= 4
#        x +   y       >= 1
#        x, y, z binary

import gurobipy as gp
from gurobipy import GRB

try:
    # Create a new model
    m = gp.Model("mip1")

    # Create variables
    x = m.addVar(vtype=GRB.BINARY, name="x")
    y = m.addVar(vtype=GRB.BINARY, name="y")
    z = m.addVar(vtype=GRB.BINARY, name="z")

    # Set objective
    m.setObjective(x + y + 2 * z, GRB.MAXIMIZE)

    # Add constraint: x + 2 y + 3 z <= 4
    m.addConstr(x + 2 * y + 3 * z <= 4, "c0")

    # Add constraint: x + y >= 1
    m.addConstr(x + y >= 1, "c1")

    # Optimize model
    m.optimize()

    for v in m.getVars():
        print(f"{v.VarName} {v.X:g}")

    print(f"Obj: {m.ObjVal:g}")

except gp.GurobiError as e:
    print(f"Error code {e.errno}: {e}")

except AttributeError:
    print("Encountered an attribute error")

In [ ]:
import numpy as np
from scipy.stats import entropy

# Function to calculate variance
def calculate_variance(water_distribution):
    mean = np.mean(water_distribution)
    variance = np.mean((water_distribution - mean) ** 2)
    return variance

# Function to calculate Gini coefficient
def calculate_gini(water_distribution):
    n = len(water_distribution)
    sorted_distribution = np.sort(water_distribution)
    cumulative_sum = np.cumsum(sorted_distribution)
    gini = (2 * np.sum((np.arange(1, n + 1) * sorted_distribution)) - (n + 1) * np.sum(sorted_distribution)) / (n * np.sum(sorted_distribution))
    return gini

# Function to calculate entropy
def calculate_entropy(water_distribution):
    total_water = np.sum(water_distribution)
    proportions = water_distribution / total_water
    return entropy(proportions)

# Function to calculate Herfindahl-Hirschman Index (HHI)
def calculate_hhi(water_distribution):
    total_water = np.sum(water_distribution)
    proportions = water_distribution / total_water
    hhi = np.sum(proportions ** 2)
    return hhi

# Example distributions
distribution_even = np.array([2, 2, 2, 2, 2])  # Even distribution
distribution_concentrated = np.array([9, 0.25, 0.25, 0.25, 0.25])  # Concentrated distribution

# Calculate metrics for even distribution
print("Even Distribution:")
print("Variance:", calculate_variance(distribution_even))
print("Gini Coefficient:", calculate_gini(distribution_even))
print("Entropy:", calculate_entropy(distribution_even))
print("HHI:", calculate_hhi(distribution_even))

print("\nConcentrated Distribution:")
# Calculate metrics for concentrated distribution
print("Variance:", calculate_variance(distribution_concentrated))
print("Gini Coefficient:", calculate_gini(distribution_concentrated))
print("Entropy:", calculate_entropy(distribution_concentrated))
print("HHI:", calculate_hhi(distribution_concentrated))

In [4]:
import re


def excel_cell_to_indices(cell_ref):
    """
    Convert an Excel-style cell reference (e.g., 'B2') to row and column indices.
    1-based indexing is used, where the first row is 1 and the first column is 1.

    Parameters:
    cell_ref (str): The Excel-style cell reference (e.g., 'B2').

    Returns:
    tuple: A tuple containing the row and column indices (row, col).
    """
    match = re.match(r"([A-Z]+)([0-9]+)", cell_ref)
    if not match:
        raise ValueError(f"Invalid cell reference: {cell_ref}")

    col_str, row_str = match.groups()

    # Convert column letters to a number (e.g., 'A' -> 1, 'B' -> 2, ..., 'AA' -> 27)
    col_num = 0
    for char in col_str:
        col_num = col_num * 26 + (ord(char.upper()) - ord("A") + 1)

    row_num = int(row_str)

    return row_num, col_num

excel_cell_to_indices('A1')

(1, 1)

In [2]:
from win32com.client import Dispatch

def export_to_excel(
    df,
    workbook_path,
    sheet_name,
    include_columns=True,
    include_index=False,
    start_position="A1",
):
    """
    Exports a pandas DataFrame to an Excel sheet using win32com.

    Parameters:
    df (pd.DataFrame): The DataFrame to export.
    workbook_path (str): The full path to the Excel workbook.
    sheet_name (str): The name of the sheet where data will be written.
    include_columns (bool): Whether to include the DataFrame's column headers. Default is True.
    include_index (bool): Whether to include the DataFrame's row indices. Default is False.
    start_position (str): The starting cell in Excel (e.g., 'B2'). Default is 'A1'.

    Returns:
    None
    """
    # Convert the start_position (e.g., 'B2') to row and column indices
    start_row, start_col = excel_cell_to_indices(start_position)

    xlApp = Dispatch("Excel.Application")
    xlApp.Visible = 1  # Make Excel visible

    xlApp.Workbooks.Open(workbook_path)

    sheet = xlApp.ActiveWorkbook.Sheets(sheet_name)
    sheet.select
    sheet.cells.clearcontents  # Clear existing contents in the sheet

    # Adjust the DataFrame based on whether to include index
    if include_index:
        df_to_export = df.reset_index()
    else:
        df_to_export = df

    # Determine the number of rows and columns
    num_rows, num_cols = df_to_export.shape

    # Write column headers if specified
    if include_columns:
        col_headers = list(df_to_export.columns)
        sheet.Range(
            sheet.Cells(start_row, start_col),
            sheet.Cells(start_row, start_col + num_cols - 1),
        ).Value = col_headers
        start_row += 1  # Move to the next row for data

    # Write the DataFrame values
    sheet.Range(
        sheet.Cells(start_row, start_col),
        sheet.Cells(start_row + num_rows - 1, start_col + num_cols - 1),
    ).Value = list(df_to_export.values)

    xlApp.ActiveWorkbook.RefreshAll()  # Refresh all (e.g., pivot tables)
    xlApp.ActiveWorkbook.Save()  # Save the workbook


In [5]:
import pandas as pd
df = pd.DataFrame({'A': [1, 2, 3], 'B': [4, 5, 6]})
export_to_excel(df, r"C:\Users\86151\Desktop\workbook.xlsx", "Sheet1", include_columns=True, include_index=True, start_position="I4")

In [1]:
import requests
import json
import gzip
import datetime

url = "https://pfsc.agri.cn/api/priceQuotationController/pageList?key=&order="

headers = {
    "accept": "application/json, text/plain, */*",
    "accept-language": "en-US,en;q=0.8",
    "content-type": "application/json;charset=UTF-8",
    "sec-ch-ua": '"Chromium";v="130", "Brave";v="130", "Not?A_Brand";v="99"',
    "sec-ch-ua-mobile": "?0",
    "sec-ch-ua-platform": '"Windows"',
    "sec-fetch-dest": "empty",
    "sec-fetch-mode": "cors",
    "sec-fetch-site": "same-origin",
    "sec-gpc": "1",
    "Referer": "https://pfsc.agri.cn/"
}

# First request to get total number of records
payload = {
    "pageNum": 1,
    "pageSize": 1,
    "marketId": "",
    "provinceCode": "",
    "pid": "",
    "varietyId": ""
}

response = requests.post(url, headers=headers, json=payload, verify=False)
total_records = response.json()['content']['total']

# Second request to get all records
payload['pageSize'] = total_records

response = requests.post(url, headers=headers, json=payload, verify=False)
data_list = response.json()['content']['list']

# Generate a better file name with timestamp
filename = datetime.datetime.now().strftime("data_%Y%m%d_%H%M%S.json.gz")

# Save the list directly to a compressed file
with gzip.open(filename, 'wt', encoding='utf-8') as f_out:
    json.dump(data_list, f_out, ensure_ascii=False, indent=4)

c:\Users\86151\miniconda3\envs\notebook\Lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host '127.0.0.1'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
c:\Users\86151\miniconda3\envs\notebook\Lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host '127.0.0.1'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
